# Bi-LSTM pour l'étiquetage morpho-syntaxique
Application du tutoriel https://nlpforhackers.io/lstm-pos-tagger-keras/

In [4]:
import nltk
nltk.download("treebank")
 
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


[nltk_data] Downloading package treebank to C:\Users\Amal
[nltk_data]     Zouaq\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


Tagged sentences:  3914
Tagged words: 100676


In [5]:
import numpy as np
 
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
# Let's see how a sequence looks
 
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [6]:
from sklearn.model_selection import train_test_split
 
 
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [7]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [8]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[7013, 9180, 1812, 6106, 211, 2935, 6694, 476, 3112, 7935, 8961, 5172, 4156, 932, 781]
[724, 704, 716, 5821, 9777, 6246, 2171, 334, 9328, 1655, 7787, 2122, 257, 3660, 1812, 5393, 9315, 9816, 4703, 2974, 4545, 9092, 8018, 7696, 6329, 4703, 9229, 2935, 4415, 3086, 1812, 3706, 4932, 4156, 2530, 353, 781]
[45, 15, 45, 26, 26, 2, 26, 26, 7, 26, 26, 19, 34, 15, 29]
[32, 42, 26, 26, 7, 26, 26, 44, 26, 6, 19, 13, 3, 19, 45, 37, 37, 15, 42, 37, 15, 32, 42, 45, 15, 42, 12, 2, 13, 23, 45, 32, 18, 34, 37, 15, 29]


In [9]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

271


In [10]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

Using TensorFlow backend.


[7013 9180 1812 6106  211 2935 6694  476 3112 7935 8961 5172 4156  932
  781    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [11]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 271, 128)          1290624   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_1 (Activation)    (None, 271, 47)           0         
Total params: 2,103,215
Trainable params: 2,103,215
Non-trainable params: 0
_________________________________________________________________


In [12]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [13]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [14]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

C:\Users\Amal Zouaq\Anaconda3\envs\keras_env\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 142s 57ms/step - loss: 1.2233 - accuracy: 0.8588 - val_loss: 0.3586 - val_accuracy: 0.9086
Epoch 2/40
2504/2504 [==============================] - 215s 86ms/step - loss: 0.3329 - accuracy: 0.9087 - val_loss: 0.3098 - val_accuracy: 0.9114
Epoch 3/40
2504/2504 [==============================] - 259s 103ms/step - loss: 0.3102 - accuracy: 0.9138 - val_loss: 0.2978 - val_accuracy: 0.9191
Epoch 4/40
2504/2504 [==============================] - 269s 108ms/step - loss: 0.2990 - accuracy: 0.9173 - val_loss: 0.2888 - val_accuracy: 0.9193
Epoch 5/40
2504/2504 [==============================] - 281s 112ms/step - loss: 0.2902 - accuracy: 0.9174 - val_loss: 0.2803 - val_accuracy: 0.9200
Epoch 6/40
2504/2504 [==============================] - 288s 115ms/step - loss: 0.2820 - accuracy: 0.9189 - val_loss: 0.2727 - val_accuracy: 0.9226
Epoch 7/40
2504/2504 [==============================] - 285s 114ms/

In [15]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

783/783 [==============================] - 32s 41ms/step
accuracy: 99.09327626228333


In [16]:
test_samples = [
    "running is very important for me .".split(),
    "I was running every day for a month .".split()
]
print(test_samples)

[['running', 'is', 'very', 'important', 'for', 'me', '.'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


In [17]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[4791 8201 6015 4500   13 3997  781    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [18]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[1.1312649e-02 1.3676315e-03 7.5726084e-06 ... 3.2171019e-04
   9.4471119e-02 1.5006609e-02]
  [3.2329361e-05 4.1634985e-03 3.7154343e-04 ... 9.3281694e-04
   5.7492484e-03 2.2881287e-03]
  [2.0794707e-06 9.6919978e-01 4.5335910e-04 ... 3.5902788e-04
   2.7558289e-04 1.0302132e-03]
  ...
  [9.9993587e-01 3.5758596e-10 2.5038858e-07 ... 1.6883289e-08
   3.1530678e-10 7.7706254e-08]
  [9.9989414e-01 5.1052124e-10 4.6374680e-07 ... 3.0207861e-08
   4.6220910e-10 1.3385413e-07]
  [9.9984145e-01 6.9094658e-10 7.6077265e-07 ... 4.8858876e-08
   6.4180855e-10 2.1360246e-07]]

 [[2.6475548e-04 1.5147317e-04 3.8425822e-04 ... 5.6175991e-06
   3.1893787e-06 4.3953173e-03]
  [1.4820204e-06 2.6480915e-04 1.0682871e-06 ... 5.3329302e-08
   2.8577315e-06 1.7745921e-04]
  [5.5112811e-05 4.0267401e-05 5.1188320e-09 ... 5.1641388e-07
   8.9390324e-03 3.4340433e-04]
  ...
  [9.9993587e-01 3.5760711e-10 2.5040745e-07 ... 1.6885609e-08
   3.1535369e-10 7.7697067e-08]
  [9.9989414e-01 5.1055138e-10 4.637

In [19]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [20]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['VBG', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

In [21]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [22]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 271, 128)          1290624   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_2 (Activation)    (None, 271, 47)           0         
Total params: 2,103,215
Trainable params: 2,103,215
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

C:\Users\Amal Zouaq\Anaconda3\envs\keras_env\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2504 samples, validate on 627 samples
Epoch 1/40
2504/2504 [==============================] - 268s 107ms/step - loss: 1.1602 - accuracy: 0.9021 - ignore_accuracy: 0.0291 - val_loss: 0.3754 - val_accuracy: 0.9114 - val_ignore_accuracy: 0.1228
Epoch 2/40
2504/2504 [==============================] - 266s 106ms/step - loss: 0.3380 - accuracy: 0.9084 - ignore_accuracy: 0.0725 - val_loss: 0.3252 - val_accuracy: 0.9072 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
2504/2504 [==============================] - 265s 106ms/step - loss: 0.3154 - accuracy: 0.9097 - ignore_accuracy: 0.0999 - val_loss: 0.3039 - val_accuracy: 0.9191 - val_ignore_accuracy: 0.1367
Epoch 4/40
2504/2504 [==============================] - 267s 107ms/step - loss: 0.3047 - accuracy: 0.9162 - ignore_accuracy: 0.1326 - val_loss: 0.3083 - val_accuracy: 0.9190 - val_ignore_accuracy: 0.1374
Epoch 5/40
2504/2504 [==============================] - 266s 106ms/step - loss: 0.2959 - accuracy: 0.9170 - ignore_accuracy: 0.1356 -

In [24]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['NNS', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

In [26]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))


783/783 [==============================] - 52s 66ms/step
ignore_accuracy: 90.68601727485657


In [27]:
print(f"{model.metrics_names[2]}: {scores[2] * 100}")

ignore_accuracy: 90.68601727485657
